### **Imports**

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

from robolab_turtlebot import Turtlebot
from TurtleControllers import TurtlebotController
from TurtleVision import TurtlebotVision
from TurtleMap import TurtlebotMap
from TurtleICP import TurtlebotICP
import TurtleUtils
import time
import loggingSetup
import logging
logger = logging.getLogger("robot")

  ### **Initialization**

In [ ]:
turtle = Turtlebot(rgb=True, pc=True)
turtle_controller = TurtlebotController(turtle, rate=40)
turtle_vision = TurtlebotVision(turtle)
turtle_map_yellow = TurtlebotMap()  # Map for yellow points in images
turtle_map_purple = TurtlebotMap()  # Map for purple points in images
turtle_icp = TurtlebotICP()

### **FSM**

In [ ]:
import abc
from typing import Dict
from collections import namedtuple

class State(abc.ABC):
    name = "default"

    def __init__(self, automat: "Automat"):
        self.automat = automat

    def execute(self) -> "State":
        print(f"Executing {self.name}")
        self.action()
        return self.get_next_state()

    @abc.abstractmethod
    def action(self):
        pass

    @abc.abstractmethod
    def get_next_state(self):
        pass

class End(State):
    name = "End"

    def get_next_state(self):
        return None

    def action(self):
        print("Done")

class Move_PG(State):
    name = "Move_PG"

    def action(self):
        self.automat.turtle_controller.move_to(self.automat.memory.P_PG, relative=False)

    def get_next_state(self):
        return self.automat.states["Scan_T1"]
    
class Move_G(State):
    name = "Move_G"

    def action(self):
        self.automat.turtle_controller.move_to(self.automat.memory.P_G, relative=False)

    def get_next_state(self):
        return self.automat.states["End"]

class Scan_T1(State):
    name = "Scan_T1"

    def action(self):
        """
        Find garage entrance
        """
        PURPLE_MIN_SIZE = 15
        YELLOW_MIN_SIZE = 20

        def reg_mean(regs):
            s, n = 0.0, len(regs) * 2
            for reg in regs:
                s += reg[0] + reg[1]
            return s / n

        def reg_mid(reg):
            return (reg[0] + reg[1]) / 2
        
        turtle_controller = self.automat.turtle_controller
        turtle_vision = self.automat.turtle_vision

        # Find Garage center
        while True:

            # Find purple stripes
            regs_purple = turtle_vision.get_regions(color = "purple",
                                                minimal_size=PURPLE_MIN_SIZE)

            # Find yellow stripe inbetween purple stripes
            regs_yellow = turtle_vision.get_regions(color = "yellow",
                                                    minimal_size=YELLOW_MIN_SIZE)
            
            # Want to see two purple pillars and a yellow region inbetween
            while len(regs_purple) < 2 or len(regs_yellow) == 0:
                turtle_controller.cmd_velocity(angular = 0.5)
                regs_purple = turtle_vision.get_regions(color = "purple",
                                                minimal_size=PURPLE_MIN_SIZE)
                regs_yellow = turtle_vision.get_regions(color = "yellow",
                                                    minimal_size=YELLOW_MIN_SIZE)
                turtle_controller.rate.sleep()
                
            # Want to look at the center of the garage entrance
            purple_mid = reg_mean(regs_purple)
            err = purple_mid - turtle_vision.img_width // 2

            if abs(err) < 10:
                # Garage entrance found, stop!
                break
            else:
                if err < 0.0:
                    err = np.clip(err, -turtle_controller.max_yaw_rate, -turtle_controller.min_yaw_rate)
                else:
                    err = np.clip(err, turtle_controller.min_yaw_rate, turtle_controller.max_yaw_rate)
                turtle_controller.cmd_velocity(linear = 0, angular = -err)
                
            turtle_controller.rate.sleep()
            
        turtle_controller.cmd_velocity(linear = 0, angular = 0)

    def get_next_state(self):
        def generate_path(turtle_vision, visualize = False):
            turtle_controller.reset_odometry()
            P1, P2, P_G, P_PG, P_GOAL = turtle_vision.garage_entry_waypoints()

            if visualize:
                fig, ax = plt.subplots(1)
                TurtleUtils.visualize_garage(ax, P1, P2, P_G, P_PG, P_GOAL)
            return P_PG, P_GOAL
        
        turtle_vision = self.automat.turtle_vision
        P_PG, P_GOAL = generate_path(turtle_vision, visualize = False)

        P_PG_prev = self.automat.memory.P_PG
        self.automat.memory.P_PG = P_PG
        self.automat.memory.P_GOAL = P_GOAL

        if P_PG_prev is None:
            return self.automat.states["Move_PG"]
        return self.automat.states["Move_G"]

class Task1(State):
    name = "Task1"

    def action(self):
        self.automat.turtle_controller.reset_odometry()
        time.sleep(0.3)

    def get_next_state(self):
        return self.automat.states["Scan_T1"]
    
class Move_T2(State):
    name = "Move_T2"

    def action(self):
        garage = self.automat.memory.garage

        pt, idx, should_scan = None, None, None
        if self.automat.memory.t2_idx is None:
            pt, idx, should_scan = garage.closest_waypoint(
                self.automat.turtle_controller.get_odometry())
        else:
            prev_idx = self.automat.memory.t2_idx
            print(f"Old point index: {prev_idx}, New point index: {idx}")
            pt, idx, should_scan = \
                garage.closest_waypoint(self.automat.memory.t2_idx)
        
        # No need to go to scanning location if scanning is not required
        if not should_scan and idx in [6,7,12]:
            self.automat.turtle_controller.move_to(pt, relative=False)

        # Book keeping
        self.automat.memory.t2_idx = idx
        self.automat.memory.should_scan = should_scan

    def get_next_state(self):
        if self.automat.memory.should_scan:
            self.automat.memory.t2_idx = None
            return self.automat.states["Task2"]
        return self.automat.states["Move_T2"]

class Scan_T2(State):
    name = "Scan_T2"

    def action(self):
        r2p = False
        current_yaw = self.automat.turtle_controller.get_odometry()[1]
        angles = np.linspace(current_yaw, current_yaw + 2 * np.pi, 12)
        for angle in angles:
            # Scan environment
            yellow_points = self.automat.turtle_vision.sample_garage(color="yellow", r2p=r2p)
            purple_points = self.automat.turtle_vision.sample_garage(sampling_step=3,
                                                                     color="purple", minimal_size=8, r2p=r2p)

            # Current odometry
            odom = turtle_controller.get_odometry()

            # Add points to maps
            self.automat.turtle_map_yellow.add_points(yellow_points, odom)
            self.automat.turtle_map_purple.add_points(purple_points, odom)

            # Rotate to next position
            self.automat.turtle_controller.rotate(angle, relative=False, tol=0.1)
            time.sleep(1.0)

    def get_next_state(self):
        use_task1 = turtle_map_purple.cluster_count >= 2
        if use_task1:
            return self.automat.states["Task1"]
        
        self.automat.turtle_map_purple.reset()

        # Fit garage model using ICP
        yellow_downsampled = self.automat.turtle_map_yellow.points_downsampled
        yellow_downsampled = TurtleUtils.robot2plt_numpy(yellow_downsampled)
        opt = turtle_icp.optimize(yellow_downsampled)
        self.automat.memory.garage = opt.garage

        # Visualization
        TurtleUtils.plot_fitted_garage(
            self.automat.memory.garage,
            TurtleUtils.robot2plt_numpy(self.automat.turtle_map_purple.points_filtered)
        )

        return self.automat.states["Move_T2"]

class Task2(State):
    name = "Task2"

    def action(self):
        pass

    def get_next_state(self):
        return self.automat.states["Scan_T2"]

class Automat:
    def __init__(self, states: list = None):
        self.states: Dict[str, State] = {}
        for state_cls in states:
            state = state_cls(self)
            self.states[state.name] = state
            assert state.name != "default", f"Name for state {state_cls} is not set"

        self.state: State = list(self.states.values())[0]

        attributes = ["garage", "should_scan", "t2_idx", "P_PG", "P_G"]] 
        Memory = namedtuple("Memory", attributes)
        self.memory = Memory(None, True, 100, None, None)

    def execute(self):
        while self.state:
            self.state = self.state.execute()


class Pycomat(Automat):
    def __init__(self):
        super().__init__([Task2, Scan_T2, Move_T2, Task1, Scan_T1, Move_PG, Move_G, End])
        self.turtle = Turtlebot(rgb=True, pc=True)
        self.turtle_controller = TurtlebotController(self.turtle, rate=40)
        self.turtle_vision = TurtlebotVision(self.turtle)
        self.turtle_map_yellow = TurtlebotMap()  # Map for yellow points in images
        self.turtle_map_purple = TurtlebotMap()  # Map for purple points in images
        self.turtle_icp = TurtlebotICP()

logger.info("Starting run")
automat = Pycomat()
automat.execute()